## Alcholic Anonymous News articles extraction from The Independent Bangladesh. Data Extraction of following parameters
- Headline
- Description
- Author
- Published_Date
- News
- URL
- Keywords
- Summary

### Importing the necessary Libraries

In [1]:
from selenium.webdriver.chrome.options import Options # enables options in web browser
from selenium import webdriver # web-based automation tool for Python
from newspaper import Article # Article scraping & curation
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys, time #  System-specific parameters and functions

### Creating Empty lists for Alcholic Anonymous News Articles parameters data to be extracted

In [2]:
headlines, descriptions, dates, authors, news, keywords, summaries, urls = [], [], [], [], [], [], [], []

### Finding the total no.of.pages by total no.of articles from google search results¶

In [3]:
keyword = 'Alcoholics Anonymous site:m.theindependentbd.com'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

soup = BeautifulSoup(get(url).text, 'lxml')
try:
    # Extracts the digits if it the resulted number without comma ','. eg: About 680 results (0.23 seconds)
    max_pages = round([int(s) for s in soup.select_one('div#resultStats').text.split() if s.isdigit()][0]/10)
    max_pages = max_pages + 1
except:
    # Extracts the digits if it the resulted number without comma ','. eg: About 1,080 results (0.23 seconds)
    max_pages = round(int(''.join(i for i in soup.select_one('div#resultStats').text if i.isdigit()))/10)
    max_pages = max_pages + 1

### Iterates max_pages value through while loop. Scraping the Articles urls

In [4]:
options = Options()
options.headless = True
browser = webdriver.Chrome(options=options)
browser.get(url)

index = 0

while True:
    try:
        index +=1
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        if index == max_pages:
            break
        browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
        time.sleep(2)
        sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
        sys.stdout.flush()
    except:
        pass
    
browser.quit()

### To remove duplicates urls entries in the list by executing below line

In [5]:
urls = list(dict.fromkeys(urls))
print(len(urls), type(urls))

3 <class 'list'>


### Iterates urls through for loop. Scraping the Articles with above parameters

In [6]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease 
        soup = BeautifulSoup(get(url).text, 'lxml')
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        
        # Extracts the Headlines
        try:
            try:
                headlines.append(article.title.strip())
            except:
                headlines.append(soup.select_one('title').text.strip())
        except:
            headlines.append(None)
            
        # Extracts the Descriptions    
        try:
            try:
                descriptions.append(soup.select_one('meta[name="description"]')['content'].strip().replace('\n', ' '))
            except:
                descriptions.append(article.meta_description.strip())
        except:
            descriptions.append(None)
            
        # Extracts the Authors
        try:
            try:
                authors.append(soup.select_one('meta[name="Author"]')['content'].strip())
            except:
                authors.append(article.authors.strip())
        except:
            authors.append(None)
        
        # Extracts the published dates
        try:
            try:
                dates.append(soup.select_one('#news_update_time').text.strip())
            except:
                dates.append(str(article.publish_date))
        except:
            dates.append(None)
            
        # Extracts the news articles
        try:
            try:
                news.append(soup.select_one('.newsDtl').text.replace('\n', '').strip())
            except:
                news.append(article.text.strip())
        except:
            news.append(None)
            
        # Extracts Keywords and Summaries    
        try:
            keywords.append(article.keywords)
            summaries.append(article.summary)
        except:
            keywords.append(None)
            summaries.append(None)
            
    except:
        headlines.append(None)
        descriptions.append(None)
        authors.append(None)
        dates.append(None)
        news.append(None)
        keywords.append(None)
        summaries.append(None)

    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

Wall time: 8.4 sheindependentbd.com/printversion/details/123691


### Checking Array Length of each list to create DataFrame

In [7]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(news), len(keywords), len(summaries), len(urls))

3 3 3 3 3 3 3 3


### Creating a csv file after checking array length and droping the missing values from the dataset

In [8]:
tbl = pd.DataFrame({'Headlines' : headlines,
                    'Descriptions' : descriptions,
                    'Authors' : authors,
                    'Published_Dates' : dates, 
                    'Articles' : news,
                    'Keywords' : keywords,
                    'Summaries' : summaries,})
#tbl = tbl.dropna()
tbl.to_csv('The_Independent_Bangladesh.csv', index=False)
tbl.head()

,Headlines,Descriptions,Authors,Published_Dates,Articles,Keywords,Summaries
0,Laughing out loud,Two police officers crash their car into a tre...,The Independent,"21 November, 2017 00:00 00 AM",Two police officers crash their car into a tre...,"[laughing, blast, need, look, man, eat, theyre...","You don’t need make-up, Jane.\nThat is so swee..."
1,Google outlines steps to tackle workplace hara...,Google on Thursday outlined changes to its han...,The Independent,"10 November, 2018 09:38:24 AM",Google on Thursday outlined changes to its han...,"[sexual, events, google, according, outlines, ...","Google will provide “more granularity,” regard..."
2,Human chain formed at JU protesting bomb threat,"Teachers, students and staff of Jahangirnagar ...",The Independent,"15 November, 2017 00:00 00 AM","Teachers, students and staff of Jahangirnagar ...","[vicechancellor, chain, bomb, students, human,...","Teachers, students and staff of Jahangirnagar ..."
